The purpose of this notebook is to see how training on half of the classes affects the accuracy.

First, check the accuracy of the flat classifier relative to the baseline.

Second, check the accuracy of the hierarchical classifier in case it did better (probably not due to high-level classes being learnt poorly).

Third, check whether training at a higher level of the hierarchy works better. (This will require training at a coarse level and excluding some classes.)

In [1]:
import pathlib

import numpy as np
import pandas as pd

import hier
import metrics

In [2]:
full_dir = '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-03-31-inat21mini/flat_softmax-lr-0.01-b-64-wd-0.0003-ep-20'

# Subset of classes, leaf nodes, flat softmax.
partition_dirs = {
    7: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d7_n2_i0_c',
    6: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d6_n2_i0_c',
    5: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d5_n2_i0_c',
    4: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d4_n2_i0_c',
}

# Subset of classes, internal nodes, flat softmax.
partition_coarse_dirs = {
    (7, 6): '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-11-inat21mini-partition/d7_n2_i0_c-max_depth_6/',
    (6, 5): '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-11-inat21mini-partition/d6_n2_i0_c-max_depth_5/',
}

# Subset of classes, leaf nodes, hierarchical loss.
hier_partition_dirs = {
    7: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d7_n2_i0_c-descendant_softmax-bal',
    6: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d6_n2_i0_c-descendant_softmax-bal',
    5: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d5_n2_i0_c-descendant_softmax-bal',
    4: '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d4_n2_i0_c-descendant_softmax-bal',
}

partition_subtrees = {
    7: 'partition_d7_n2_i0_c',
    6: 'partition_d6_n2_i0_c',
    5: 'partition_d5_n2_i0_c',
    4: 'partition_d4_n2_i0_c',
}

coarse_subtrees = {
    6: 'max_depth_6',
    5: 'max_depth_5',
    4: 'max_depth_4',
    3: 'max_depth_5',
}

# partition_dir = '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-20-inat21mini-partition/d7_n2_i0_c'
# hier_dir = '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-11-inat21mini-partition/d7_n2_i0_c-hier'
# coarse_dir = '/mnt/ssd1/projects/2022-01-hierarchical/experiments/2022-04-11-inat21mini-partition/d7_n2_i0_c-max_depth_6/'

In [3]:
partition_level = 6
coarse_level = partition_level - 1

In [4]:
with open('resources/hierarchy/inat21.csv') as f:
    tree, tree_names = hier.make_hierarchy_from_edges(hier.load_edges(f))

with open(f'resources/subtree/inat21_{partition_subtrees[partition_level]}.csv') as f:
    train_subtree, train_subtree_names = hier.make_hierarchy_from_edges(hier.load_edges(f))

with open(f'resources/subtree/inat21_{coarse_subtrees[coarse_level]}.csv') as f:
    eval_subtree, eval_subtree_names = hier.make_hierarchy_from_edges(hier.load_edges(f))

In [5]:
outputs = np.load(pathlib.Path(full_dir) / 'predictions/output-epoch-0020.pkl', allow_pickle=True)
pred_full = outputs['pred']
gt = outputs['gt']

In [6]:
outputs = np.load(pathlib.Path(partition_dirs[partition_level]) / 'predictions/output-epoch-0020.pkl', allow_pickle=True)
# Change to full node index.
train_node_subset = hier.find_subset_index(tree_names, train_subtree_names)
pred_fold = {k: train_node_subset[v] for k, v in outputs['pred'].items()}

In [7]:
# Project everything to evaluation subtree.
# Not *strictly* correct to project leaf prediction, but good enough for now.

eval_node_subset = hier.find_subset_index(tree_names, eval_subtree_names)
eval_projection = hier.find_projection(tree, eval_node_subset)

gt_proj = eval_projection[gt]
pred_full_proj = {k: eval_projection[v] for k, v in pred_full.items()}
pred_fold_proj = {k: eval_projection[v] for k, v in pred_fold.items()}

In [8]:
# Compare (projected) accuracy of full- and subset-trained models.
np.mean(gt_proj == pred_full_proj['leaf']), np.mean(gt_proj == pred_fold_proj['leaf'])

(0.69642, 0.49967)

In [9]:
def split_by_value(x):
    order = np.argsort(x)
    values, first_index = np.unique(x[order], return_index=True)
    return values, np.split(order, first_index[1:])

In [10]:
values, groups = split_by_value(gt)
seen = np.concatenate([groups[i] for i in np.flatnonzero(np.isin(values, train_node_subset))])
unseen = np.concatenate([groups[i] for i in np.flatnonzero(np.isin(values, train_node_subset, invert=True))])
len(seen), len(unseen)

(50810, 49190)

In [12]:
# Compare (projected) accuracy of full- and subset-trained models on seen and unseen classes.

np.mean(gt_proj[seen] == pred_full_proj['leaf'][seen]), np.mean(gt_proj[seen] == pred_fold_proj['leaf'][seen])

(0.6904939972446369, 0.7082464081873647)

In [13]:
np.mean(gt_proj[unseen] == pred_full_proj['leaf'][unseen]), np.mean(gt_proj[unseen] == pred_fold_proj['leaf'][unseen])

(0.7025411669038423, 0.28422443586094737)

In [38]:
# Compare against training at coarse level.

outputs = np.load(pathlib.Path(partition_coarse_dirs[partition_level, coarse_level]) / 'predictions/output-epoch-0020.pkl', allow_pickle=True)
pred_coarse = {k: eval_node_subset[v] for k, v in outputs['pred'].items()}
pred_coarse_proj = {k: eval_projection[v] for k, v in pred_coarse.items()}

print('overall: {:.1%}, {:.1%}'.format(
    np.mean((gt_proj == pred_fold_proj['leaf'])),
    np.mean((gt_proj == pred_coarse_proj['leaf']))))
print('seen:    {:.1%}, {:.1%}'.format(
    np.mean((gt_proj == pred_fold_proj['leaf'])[seen]),
    np.mean((gt_proj == pred_coarse_proj['leaf'])[seen])))
print('unseen:  {:.1%}, {:.1%}'.format(
    np.mean((gt_proj == pred_fold_proj['leaf'])[unseen]),
    np.mean((gt_proj == pred_coarse_proj['leaf'])[unseen])))

overall: 50.0%, 51.2%
seen:    70.8%, 66.8%
unseen:  28.4%, 35.2%


In [39]:
# Majority-inference instead.
print('overall: {:.1%}, {:.1%}'.format(
    np.mean((gt_proj == pred_fold_proj['majority'])),
    np.mean((gt_proj == pred_coarse_proj['majority']))))
print('seen:    {:.1%}, {:.1%}'.format(
    np.mean((gt_proj == pred_fold_proj['majority'])[seen]),
    np.mean((gt_proj == pred_coarse_proj['majority'])[seen])))
print('unseen:  {:.1%}, {:.1%}'.format(
    np.mean((gt_proj == pred_fold_proj['majority'])[unseen]),
    np.mean((gt_proj == pred_coarse_proj['majority'])[unseen])))

overall: 40.4%, 41.9%
seen:    59.3%, 56.0%
unseen:  20.8%, 27.3%


In [17]:
np.mean(gt_proj[seen] == pred_full_proj['majority'][seen]), np.mean(gt_proj[seen] == pred_coarse_proj['majority'][seen])

(0.5488880141704389, 0.5602637276126746)

In [19]:
# Check unseen-accuracy from coarse training.
np.mean(gt_proj[unseen] == pred_fold_proj['majority'][unseen]), np.mean(gt_proj[unseen] == pred_coarse_proj['majority'][unseen])

(0.20815206342752593, 0.2732872535068103)

In [14]:
# Check whether the hierarchical model did better on the unseen classes.
outputs = np.load(pathlib.Path(hier_partition_dirs[partition_level]) / 'predictions/output-epoch-0020.pkl', allow_pickle=True)
pred_hier = {k: train_node_subset[v] for k, v in outputs['pred'].items()}
pred_hier_proj = {k: eval_projection[v] for k, v in pred_hier.items()}

In [ ]:
# About the same really.
np.mean(gt_proj[unseen] == pred_fold_proj['leaf'][unseen]), np.mean(gt_proj[unseen] == pred_hier_proj['leaf'][unseen])

In [ ]:
# How about correctness of majority prediction? Should be better.
(np.mean(metrics.correct(eval_subtree, gt_proj[unseen], pred_fold_proj['majority'][unseen])),
 np.mean(metrics.correct(eval_subtree, gt_proj[unseen], pred_hier_proj['majority'][unseen])))